# Understanding Reduction Computation

## Placing the reduce axis at the end

In [3]:
from shrimpgrad.util import prod
import itertools
import operator

class Tensor:
    def __init__(self, shape, data):
        self.shape = shape
        self.data = data
        self.strides = tuple(itertools.accumulate(reversed(shape[1:]), operator.mul, initial=1))[::-1]
    
    def permute(self, order):
        self.shape = tuple([self.shape[i] for i in order])
        self.strides = tuple([self.strides[i] for i in order])


t = Tensor((2,2,2), [1,2,3,4]*2)

# axis=0
shape = t.shape
out_shape = (1,1,2)
out = Tensor(out_shape, [0]*2)

order = tuple([i for i,s in enumerate(shape) if shape[i] == out_shape[i]] + [i for i,s in enumerate(shape) if out_shape[i] != shape[i]])
print(f"{order = }")

t.permute(order)
out.permute(order)

order = (2, 0, 1)


In [ ]:
t.shape, t.strides, out.shape, out.strides

What does it mean for the computation when the reduce axis is always in the last dimension?

In [ ]:
# Naive

In [ ]:
off=0
acc = 0
for m in range(t.shape[0]):
    moff = m * t.strides[0]
    for r in range(t.shape[1]):
        roff = r * t.strides[1]
        for c in range(t.shape[2]):
            coff = moff + roff + c*t.strides[2]
            # print(t.datax[coff])
            out.data[off] += t.data[coff]
            print(off, t.data[coff])
    off+=1
            
   
print(out.data)
# 0,0 1,1
# 2,2 3,3

In [ ]:
import numpy as np
x = np.array([1,2,3,4]*2).reshape((2,2,2))
print(x)

In [ ]:
x.transpose(order).sum((0,1), keepdims=True)